In [2]:
d ={"key":"value","key1":"value2"}

In [8]:
from box import ConfigBox

In [9]:
d2 = ConfigBox({"key1":"a","key2":'b',"ke3":'x'}) 

In [10]:
d2.key1

'a'

In [11]:
def get_product (x:int, y:int) -> int:
    return x*y

In [12]:
get_product(4,5)

20

In [13]:
get_product(x=3,y="4")

'444'

In [16]:
#from ensure import ensure_annotations